In [1]:
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import scipy.interpolate
import plotly.express as px
import plotly.graph_objects as go

In [2]:
url = 'https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/odata/ExpectativasMercadoAnuais?$format=json'
expec = pd.read_json(url)
expec = expec['value'].apply(pd.Series) #transformando coluna em dataframe
expec = expec.sort_values(by=['Data']) #ordenando

In [151]:
ipca = expec[expec['Indicador'] == 'IPCA'].copy()

In [152]:
ipca['Data'] = pd.to_datetime(ipca['Data'])

In [153]:
ipca['ano_corrente'] = pd.DatetimeIndex(ipca['Data']).year

In [154]:
ipca['DataReferencia'] = ipca['DataReferencia'].astype(float)
ipca['verica_ano_corrente'] = ipca['ano_corrente'] == ipca['DataReferencia']
ipca.set_index(['Data'], drop = True, inplace = True)

In [155]:
ipca['last_day_year'] = ipca['DataReferencia'].apply(lambda x: str(int(x))+'-'+'12' + '-' + '31') # Criando data referente ao último dia útil do ano da data de referência
ipca['last_day_year'] = pd.to_datetime(ipca['last_day_year'])

In [156]:
ipca['time_to_end'] = (ipca['last_day_year'] - ipca.index).dt.days # criando quantidade de dias entre data de divulgação e data da inflação do fim do período

In [157]:
ipca = ipca[ipca['DataReferencia'] >= ipca['ano_corrente'] ]

In [158]:
ipca = ipca[ipca['baseCalculo'] == 0]

In [159]:
# a partir do time_to_end (diferença entre data de divulgação e data do fim do período de referência), iremos
# interpolar os valores para obter as inflação referente a n * 365 dias à frente. Obtendo, portanto, a expectativa de inflação
# n horizonte à frente
lista_dfs = []
for i in pd.DataFrame(ipca.index).drop_duplicates().reset_index(drop=True).set_index(['Data']).index:
    dfzinho = ipca[ipca.index == i]
    dfzinho.sort_values(by=['DataReferencia'], inplace = True)
    list_horizons = []
    list_interpol_values = []
    lista_horizontes = dfzinho['DataReferencia'] - dfzinho['ano_corrente']

    lista_horizontes = lista_horizontes * 365
    lista_horizontes = lista_horizontes.to_list()
    lista_horizontes = [i for i in lista_horizontes if i != 0]

    try:
        y_interp = scipy.interpolate.interp1d(dfzinho['time_to_end'], dfzinho['Media'])


        for i in lista_horizontes:
            list_interpol_values.append(float(y_interp(i)))
            list_horizons.append(i/365)

    except:
        list_interpol_values.append(dfzinho['Media'][0])
        list_horizons.append(1)
    
        

    d = {'anos_a_frente':list_horizons,'expec':list_interpol_values}
    df_result = pd.DataFrame(d)
    df_result['Data'] = dfzinho.index[0]
    df_result.set_index(['Data'], inplace = True)
    #df_result['expec'] = df_result['expec'].astype(float)
    
    lista_dfs.append(df_result)

<ipython-input-159-0b3bc19ef534>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [160]:
df_a_frente = pd.concat(lista_dfs, axis = 0)

In [161]:
df_a_frente.reset_index(inplace = True)

In [162]:
df_pivot = df_a_frente.pivot(index='Data', columns='anos_a_frente', values=['expec'])


In [163]:
df_pivot.columns = df_pivot.columns.get_level_values(1)
df_pivot.columns = [str(int(i)) + ' anos à frente' for i in df_pivot.columns]


In [164]:
df_pivot.rename(columns={'1 anos à frente': '1 ano à frente'}, inplace = True)

In [166]:
fig = px.line(df_pivot, x=df_pivot.index, y= df_pivot.columns, title='Expectativa de inflação - IPCA Acumulado 12 Meses (Média Boletim Focus)')

fig.update_xaxes(
    rangeslider_visible=False,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig['data'][0]['line']['color']="#000000"
fig['data'][1]['line']['color']="blue"


fig.update_layout(
    #title="Plot Title",
    xaxis_title="date",
    yaxis_title="%",
    legend_title="")

fig.show()